In [68]:
import numpy as np
import scipy.io
import scipy.linalg
import sklearn.metrics
from sklearn.neighbors import KNeighborsClassifier


def kernel(ker, X1, X2, gamma):
    K = None
    if not ker or ker == 'primal':
        K = X1
    elif ker == 'linear':
        if X2 is not None:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1).T, np.asarray(X2).T)
        else:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1).T)
    elif ker == 'rbf':
        if X2 is not None:
            K = sklearn.metrics.pairwise.rbf_kernel(np.asarray(X1).T, np.asarray(X2).T, gamma)
        else:
            K = sklearn.metrics.pairwise.rbf_kernel(np.asarray(X1).T, None, gamma)
    return K


class TCA:
    def __init__(self, kernel_type='primal', dim=30, lamb=1, gamma=1):
        '''
        Init func
        :param kernel_type: kernel, values: 'primal' | 'linear' | 'rbf'
        :param dim: dimension after transfer
        :param lamb: lambda value in equation
        :param gamma: kernel bandwidth for rbf kernel
        '''
        self.kernel_type = kernel_type
        self.dim = dim
        self.lamb = lamb
        self.gamma = gamma

    def fit(self, Xs, Xt):
        '''
        Transform Xs and Xt
        :param Xs: ns * n_feature, source feature
        :param Xt: nt * n_feature, target feature
        :return: Xs_new and Xt_new after TCA
        '''
        X = np.hstack((Xs.T, Xt.T))
        X = X / np.linalg.norm(X, axis=0)
        m, n = X.shape
        ns, nt = len(Xs), len(Xt)
        e = np.vstack((1 / ns * np.ones((ns, 1)), -1 / nt * np.ones((nt, 1))))
        M = e * e.T
        M = M / np.linalg.norm(M, 'fro')
        H = np.eye(n) - 1 / n * np.ones((n, n))
        K = kernel(self.kernel_type, X, None, gamma=self.gamma)
        n_eye = m if self.kernel_type == 'primal' else n
        a, b = np.linalg.multi_dot([K, M, K.T]) + self.lamb * np.eye(n_eye), np.linalg.multi_dot([K, H, K.T])
        w, V = scipy.linalg.eig(a, b)
        ind = np.argsort(w)
        A = V[:, ind[:self.dim]]
        Z = np.dot(A.T, K)
        Z /= np.linalg.norm(Z, axis=0)
        Xs_new, Xt_new = Z[:, :ns].T, Z[:, ns:].T
        return Xs_new, Xt_new

    def fit_predict(self, Xs, Ys, Xt, Yt):
        '''
        Transform Xs and Xt, then make predictions on target using 1NN
        :param Xs: ns * n_feature, source feature
        :param Ys: ns * 1, source label
        :param Xt: nt * n_feature, target feature
        :param Yt: nt * 1, target label
        :return: Accuracy and predicted_labels on the target domain
        '''
        Xs_new, Xt_new = self.fit(Xs, Xt)
        clf = KNeighborsClassifier(n_neighbors=1)
        clf.fit(Xs_new, Ys.ravel())
        y_pred = clf.predict(Xt_new)
        acc = sklearn.metrics.accuracy_score(Yt, y_pred)
        return acc, y_pred

In [69]:
domains = ['caltech.mat', 'amazon.mat', 'webcam.mat', 'dslr.mat']
src, tar = 'data/' + domains[2], 'data/' + domains[3]
src_domain, tar_domain = scipy.io.loadmat(src), scipy.io.loadmat(tar)

In [70]:
print(src_domain.keys())

dict_keys(['__header__', '__version__', '__globals__', 'fts', 'labels'])


In [71]:
Xs, Ys, Xt, Yt = src_domain['fts'], src_domain['labels'], tar_domain['fts'], tar_domain['labels']
print(Xs,Xs.shape, '\n', Ys.ravel(), Ys.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] (295, 800) 
 [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 

In [72]:
tca = TCA(kernel_type='linear', dim=10, lamb=1, gamma=1)

acc, ypre = tca.fit_predict(Xs, Ys, Xt, Yt)
print(acc)
tca = TCA(kernel_type='linear', dim=30, lamb=1, gamma=1)

acc, ypre = tca.fit_predict(Xs, Ys, Xt, Yt)
print(acc)

tca = TCA(kernel_type='linear', dim=100, lamb=1, gamma=1)

acc, ypre = tca.fit_predict(Xs, Ys, Xt, Yt)
print(acc)

tca = TCA(kernel_type='linear', dim=300, lamb=1, gamma=1)

acc, ypre = tca.fit_predict(Xs, Ys, Xt, Yt)
print(acc)

0.6560509554140127
0.7707006369426752
0.7770700636942676
0.7707006369426752


In [84]:
tca_more = TCA(kernel_type='rbf', dim=10, lamb=1, gamma=1)

acc_more, _ = tca_more.fit_predict(Xs, Ys, Xt, Yt)
print(acc_more)

tca_more = TCA(kernel_type='rbf', dim=30, lamb=1, gamma=1)

acc_more, _ = tca_more.fit_predict(Xs, Ys, Xt, Yt)
print(acc_more)

tca_more = TCA(kernel_type='rbf', dim=100, lamb=1, gamma=1)

acc_more, _ = tca_more.fit_predict(Xs, Ys, Xt, Yt)
print(acc_more)

tca_more = TCA(kernel_type='rbf', dim=300, lamb=1, gamma=1)

acc_more, _ = tca_more.fit_predict(Xs, Ys, Xt, Yt)
print(acc_more)
tca_rbf = TCA(kernel_type='rbf', dim=800, lamb=1, gamma=1)

acc_rbf, _ = tca_rbf.fit_predict(Xs, Ys, Xt, Yt)
print(acc_rbf)



0.6560509554140127
0.7515923566878981
0.8407643312101911
0.7834394904458599
0.8089171974522293


In [74]:
clf = KNeighborsClassifier(n_neighbors=1)
print(Ys.ravel())
clf.fit(Xs, Ys.ravel())
y_pred = clf.predict(Xt)
acc_org = sklearn.metrics.accuracy_score(Yt, y_pred)
print(acc_org)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3
  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5  5  5  5  5
  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  9  9
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 10]
0.554140127388535


In [75]:
X = np.hstack((Xs.T, Xt.T))
X = X / np.linalg.norm(X, axis=0)
m, n = X.shape
ns, nt = len(Xs), len(Xt)
e = np.vstack((1 / ns * np.ones((ns, 1)), -1 / nt * np.ones((nt, 1))))
print('E', e.shape)
M = e * e.T
print('M', M.shape)
M = M / np.linalg.norm(M, 'fro')
H = np.eye(n) - 1 / n * np.ones((n, n))
print('H', H, H.shape)
K = kernel(kernel_type, X, None, gamma=gamma)
n_eye = m if kernel_type == 'primal' else n
print(n_eye)
print(lamb * np.eye(n_eye))
a, b = np.linalg.multi_dot([K, M, K.T]) + lamb * np.eye(n_eye), np.linalg.multi_dot([K, H, K.T])
w, V = scipy.linalg.eig(a, b)
ind = np.argsort(w)
print('w', w, w.shape)
print('V', V, V.shape)
A = V[:, ind[:dim]]
Z = np.dot(A.T, K)
print(Z, Z.shape)
'''Z /= np.linalg.norm(Z, axis=0)
Xs_new, Xt_new = Z[:, :ns].T, Z[:, ns:].T
'''

E (452, 1)
M (452, 452)
H [[ 0.99778761 -0.00221239 -0.00221239 ... -0.00221239 -0.00221239
  -0.00221239]
 [-0.00221239  0.99778761 -0.00221239 ... -0.00221239 -0.00221239
  -0.00221239]
 [-0.00221239 -0.00221239  0.99778761 ... -0.00221239 -0.00221239
  -0.00221239]
 ...
 [-0.00221239 -0.00221239 -0.00221239 ...  0.99778761 -0.00221239
  -0.00221239]
 [-0.00221239 -0.00221239 -0.00221239 ... -0.00221239  0.99778761
  -0.00221239]
 [-0.00221239 -0.00221239 -0.00221239 ... -0.00221239 -0.00221239
   0.99778761]] (452, 452)
452
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
w [           inf+0.j 9.96012937e+03+0.j 6.27181620e+03+0.j
 3.60626709e+03+0.j 3.19676544e+03+0.j 2.83222216e+03+0.j
 2.85321335e+03+0.j 2.45899630e+03+0.j 1.96600429e+03+0.j
 1.87912419e+03+0.j 1.79994175e+03+0.j 1.60600938e+03+0.j
 1.53032850e+03+0.j 1.42133327e+03+0.j 1.39982959e+03+0.j
 1.27924053e+03+0.j

'Z /= np.linalg.norm(Z, axis=0)\nXs_new, Xt_new = Z[:, :ns].T, Z[:, ns:].T\n'

In [76]:
help(scipy.linalg.eig)

Help on function eig in module scipy.linalg.decomp:

eig(a, b=None, left=False, right=True, overwrite_a=False, overwrite_b=False, check_finite=True, homogeneous_eigvals=False)
    Solve an ordinary or generalized eigenvalue problem of a square matrix.
    
    Find eigenvalues w and right or left eigenvectors of a general matrix::
    
        a   vr[:,i] = w[i]        b   vr[:,i]
        a.H vl[:,i] = w[i].conj() b.H vl[:,i]
    
    where ``.H`` is the Hermitian conjugation.
    
    Parameters
    ----------
    a : (M, M) array_like
        A complex or real matrix whose eigenvalues and eigenvectors
        will be computed.
    b : (M, M) array_like, optional
        Right-hand side matrix in a generalized eigenvalue problem.
        Default is None, identity matrix is assumed.
    left : bool, optional
        Whether to calculate and return left eigenvectors.  Default is False.
    right : bool, optional
        Whether to calculate and return right eigenvectors.  Default is Tru

In [80]:
X = np.hstack((Xs.T, Xt.T))
X = X / np.linalg.norm(X, axis=0)
m, n = X.shape
ns, nt = len(Xs), len(Xt)
e = np.vstack((1 / ns * np.ones((ns, 1)), -1 / nt * np.ones((nt, 1))))

M = e * e.T
M = M / np.linalg.norm(M, 'fro')
H = np.eye(n) - 1 / n * np.ones((n, n))
K = kernel(kernel_type, X, None, gamma=gamma)
n_eye = m if kernel_type == 'primal' else n
a, b = np.linalg.multi_dot([K, M, K.T]) + lamb * np.eye(n_eye), np.linalg.multi_dot([K, H, K.T])
w, V = scipy.linalg.eig(a, b)
w1, V1 = scipy.linalg.eig(a,b)
print('w1', w1, w1.shape)

ind = np.argsort(w)
print('w', w, w.shape)
print('V', V, V.shape)
print('V1', V1,V1.shape)
A = V[:, ind[:dim]]
Z = np.dot(A.T, K)
print(w == w1)
print(V1 == V)

w1 [           inf+0.j 9.96012937e+03+0.j 6.27181620e+03+0.j
 3.60626709e+03+0.j 3.19676544e+03+0.j 2.83222216e+03+0.j
 2.85321335e+03+0.j 2.45899630e+03+0.j 1.96600429e+03+0.j
 1.87912419e+03+0.j 1.79994175e+03+0.j 1.60600938e+03+0.j
 1.53032850e+03+0.j 1.42133327e+03+0.j 1.39982959e+03+0.j
 1.27924053e+03+0.j 1.26629989e+03+0.j 1.09228311e+03+0.j
 1.05220570e+03+0.j 1.01907609e+03+0.j 9.43046013e+02+0.j
 9.06047380e+02+0.j 8.32061726e+02+0.j 7.99673209e+02+0.j
 7.73611021e+02+0.j 7.19101925e+02+0.j 6.90363915e+02+0.j
 6.59003841e+02+0.j 6.43261385e+02+0.j 5.70288620e+02+0.j
 5.60466167e+02+0.j 5.48749390e+02+0.j 5.24114128e+02+0.j
 5.00033854e+02+0.j 4.96099934e+02+0.j 4.75517034e+02+0.j
 4.51416745e+02+0.j 4.27716722e+02+0.j 4.05417301e+02+0.j
 4.00613849e+02+0.j 3.88763959e+02+0.j 3.82014772e+02+0.j
 3.68670448e+02+0.j 3.55198751e+02+0.j 3.38713080e+02+0.j
 3.28378730e+02+0.j 3.14005964e+02+0.j 3.06837982e+02+0.j
 2.94564785e+02+0.j 2.87187501e+02+0.j 2.77215362e+02+0.j
 2.65061093